In [1]:
import pandas as pd
import time
from collections import deque
import numpy as np
pd.options.mode.chained_assignment = None

In [2]:
# read the dataframes
all_games = pd.read_pickle("data/games.pkl")
all_orders = pd.read_pickle("data/orders.pkl")
all_players = pd.read_pickle("data/players.pkl")
all_turns = pd.read_pickle("data/turns.pkl")
all_units = pd.read_pickle("data/units.pkl")

# remove duplicates
all_units = all_units.drop_duplicates()

In [25]:
countries = ['A', 'E', 'F', 'G', 'I', 'R', 'T']
pairs = [x+y for x in countries for y in countries if y > x]

# What is the effect of betrayal over wining ? 

We look at the outcome **winning** (binary) considering the treatment which can be 
- betrayer
- betrayed
- none, that we call the 'neutrals'

The first thing we need is to know when each player 'dies' and who is (are) the winner(s). 

In [70]:
def get_betrayers_and_betrayed(friendships):
    cols = [col for col in friendships.columns if np.count_nonzero(friendships[col] != 0)]
    betrayers = []
    betrayeds = []
    for c in cols: 
        tmp = friendships[c]
        values = tmp[tmp != 0].values
        if type(values[-1]) == str: 
            betrayer = values[-1]
            betrayers.append(betrayer)
            tmp = list(c)
            tmp.remove(betrayer)
            betrayeds.append(tmp[0])
            
    return betrayers, betrayeds

In [71]:
def get_winner_country(game_id):
    winner = all_players.query("game_id == @game_id & won == 1")
    return winner.country.values

In [72]:
def get_loosers(winners):
    loosers = countries.copy()
    for w in winners: loosers.remove(w)
    return loosers

In [77]:
def get_neutrals(betrayers, betrayeds):
    neutrals = countries.copy()
    for b in betrayers: 
        if b in neutrals: neutrals.remove(b)
    for b in betrayeds: 
        if b in neutrals: neutrals.remove(b)
    return neutrals

In [96]:
# load the data to analyse
games_id = np.load("data/subset2/games_id.npy")
all_friendships = np.load("data/subset2/friendships.npy", allow_pickle=True)


winners_betrayer = 0
loosers_betrayer = 0
loosers_betrayed = 0 
N = len(games_id)
for i, game_id in enumerate(games_id[:30]):
    
    # reconstruct the obtained data
    data = all_friendships[i]
    years = np.arange(1901, 1901 + data.shape[0] * 0.5, 0.5)
    friendships = pd.DataFrame(data = all_friendships[i], columns = pairs, index = years)
    
    # get labels
    winners = get_winner_country(game_id)
    loosers = get_loosers(winners)
    betrayers, betrayeds = get_betrayers_and_betrayed(friendships)
    neutrals = get_neutrals(betrayers, betrayeds)
    
    
    print("\nGame",i)
    print("Winners: ", winners, " and Loosers", loosers)
    print("Betrayers: ", betrayers)
    
    for winner in winners: 
        if winner in betrayers: winners_betrayer += 1
    for looser in loosers: 
        if looser in betrayers: loosers_betrayer += 1


Game 0
Winners:  ['T']  and Loosers ['A', 'E', 'F', 'G', 'I', 'R']
Betrayers:  ['R']

Game 1
Winners:  ['A']  and Loosers ['E', 'F', 'G', 'I', 'R', 'T']
Betrayers:  ['A']

Game 2
Winners:  ['T']  and Loosers ['A', 'E', 'F', 'G', 'I', 'R']
Betrayers:  []

Game 3
Winners:  ['E']  and Loosers ['A', 'F', 'G', 'I', 'R', 'T']
Betrayers:  []

Game 4
Winners:  ['R']  and Loosers ['A', 'E', 'F', 'G', 'I', 'T']
Betrayers:  ['R']

Game 5
Winners:  ['T']  and Loosers ['A', 'E', 'F', 'G', 'I', 'R']
Betrayers:  []

Game 6
Winners:  ['G']  and Loosers ['A', 'E', 'F', 'I', 'R', 'T']
Betrayers:  []

Game 7
Winners:  ['F' 'R']  and Loosers ['A', 'E', 'G', 'I', 'T']
Betrayers:  []

Game 8
Winners:  ['I' 'T']  and Loosers ['A', 'E', 'F', 'G', 'R']
Betrayers:  []

Game 9
Winners:  ['T']  and Loosers ['A', 'E', 'F', 'G', 'I', 'R']
Betrayers:  ['T']

Game 10
Winners:  ['T']  and Loosers ['A', 'E', 'F', 'G', 'I', 'R']
Betrayers:  ['G']

Game 11
Winners:  ['R']  and Loosers ['A', 'E', 'F', 'G', 'I', 'T']
Betr

In [97]:
print("N = ", N)
print("winners betrayer", winners_betrayer)
print("loosers_betrayer", loosers_betrayer)

N =  50
winners betrayer 5
loosers_betrayer 7
